# iKnow standalone demo

In [1]:
import iknowpy

iknow = iknowpy.iKnowEngine()

In [2]:
iknow.index("The patient did not report kidney beans, but had a lower back pain after gardening on Saturday.","en")
# iknow.index("He wasn't very happy with the cake, even though we put much effort (and no sugar) into it.","en")

In [3]:
import pprint
pp = pprint.PrettyPrinter(indent=2)
pp.pprint(iknow.m_index)

{ 'proximity': [ ((5, 7), 64),
                 ((1, 3), 64),
                 ((3, 5), 64),
                 ((3, 7), 42),
                 ((1, 5), 42),
                 ((1, 7), 32)],
  'sentences': [ { 'entities': [ { 'dominance_value': 0.0,
                                   'entity_id': 0,
                                   'index': 'the',
                                   'offset_start': 0,
                                   'offset_stop': 3,
                                   'type': 'NonRelevant'},
                                 { 'dominance_value': 333.0,
                                   'entity_id': 1,
                                   'index': 'patient',
                                   'offset_start': 4,
                                   'offset_stop': 11,
                                   'type': 'Concept'},
                                 { 'dominance_value': 1000.0,
                                   'entity_id': 2,
                                   'index':

In [4]:
# print basic parsing output
for s in iknow.m_index['sentences']:
    for e in s['entities']:
        print('<'+e['type']+'>'+e['index']+'</'+e['type']+'>', end=' ')
    print('\n')

<NonRelevant>the</NonRelevant> <Concept>patient</Concept> <Relation>did not report</Relation> <Concept>kidney beans</Concept> <Relation>but had</Relation> <NonRelevant>a</NonRelevant> <Concept>lower back pain</Concept> <Relation>after gardening on</Relation> <Concept>saturday</Concept> 



In [69]:
# now use colorama to make it look nicer
from colorama import Fore, Style

#from colorama import init
#init() # init colorama - only when running outside notebook

def highlight(text, language="en", iknow=iknowpy.iKnowEngine()):
    
    iknow.index(text, language)
    
    for s in iknow.m_index['sentences']:
        
        # first figure out where negation spans are and tag those entities
        for a in s['path_attributes']:
            
            # path attributes are expressed as positions within s['path'],
            # which in turn keys into the s['entities'] array
            for ent in range(s['path'][a['pos']], 
                             s['path'][a['pos']+a['span']-1]+1):
                if a['type']=="Negation":
                    s['entities'][ent]['colour'] = Fore.RED
                if a['type']=="Certainty":
                    s['entities'][ent]['colour'] = Fore.CYAN
                    
        for e in s['entities']:
            colour = Fore.BLACK
            style = Style.NORMAL
            
            if "colour" in e:
                colour = e["colour"]
                
            if (e['type'] == 'Concept'):
                style = Style.BRIGHT
            if (e['type'] == 'NonRelevant') | (e['type'] == 'PathRelevant'):
                style = Style.DIM
                
            print(colour + style + text[e['offset_start']:e['offset_stop']], end=' ')
            
        print("\n")


In [70]:
highlight("the Kat does not like the potatoes, but ate them anyway")

the Kat does not like the potatoes, but ate them anyway 



In [71]:
# now let's use this to parse an RSS feed
import feedparser

feed = feedparser.parse("http://newsrss.bbc.co.uk/rss/newsonline_world_edition/americas/rss.xml")

for entry in feed.entries:
    print(entry.description)

The social network has activated password reset protections and is demanding stronger passwords.
Torrential rains could bring damaging floods to the Carolinas, Georgia and Virginia.
The disgraced film mogul, jailed this year for rape and sexual assault, received the honour in 2004.
The US president relied on over-65s to win the White House in 2016 but their support this time is not assured.
Former model Amy Dorris alleges Mr Trump groped her and forcibly kissed her at the US Open in 1997.
Canadian police said both front seats were fully reclined as the car drove at 150km/h.
His book was made into the hugely successful Oscar-winning 1994 film starring Tom Hanks.
Prosecutors say Jerry Harris enticed an underage boy to produce sexually explicit videos and photos.
Military police requested a heat ray before clearing a Washington DC square in June, a major says.
The star says he won't release new music until he is released from his recording contract.
The first half of the ex-president's me

In [72]:
for entry in feed.entries:
    highlight(entry.description)

The social network has activated password reset protections and is demanding stronger passwords. 

Torrential rains could bring damaging floods to the Carolinas, Georgia and Virginia. 

The disgraced film mogul, jailed this year for rape and sexual assault, received the honour in 2004. 

The US president relied on over-65s to win the White House in 2016 but their support this time is not assured. 

Former model Amy Dorris alleges Mr Trump groped her and forcibly kissed her at the US Open in 1997. 

Canadian police said both front seats were fully reclined as the car drove at 150km/h. 

His book was made into the hugely successful Oscar-winning 1994 film starring Tom Hanks. 

Prosecutors say Jerry Harris enticed an underage boy to produce sexually explicit videos and photos. 

Military police requested a heat ray before clearing a Washington DC square in June, a major says. 

The star says he won't release new music until he is released from his recording contract. 

The first half of t